# Agents(代理) and tools(工具)

## 什么是 Agent (代理) ?

大型语言模型（LLMs）可以处理各种各样的任务，但它们通常难以处理逻辑、计算和搜索等基本任务。当在提示他们在表现不佳的领域进行生成时，往往无法直接生成我们期望的答案。

克服这个缺点的方法之一是创建一个`Agent`。`Agent`是一个使用LLM作为引擎的系统，它可以访问称为`Tools`的功能。这些`Tools`是用于执行任务的功能，同时也包含使代理能够正确使用它们的描述信息。

Agents的程序可编程为：

- 设计一系列动作(Actions)和工具(Tools)并一次性运行它们，例如`CodeAgent`。
- 逐个计划并执行操作或工具，等待每个操作的结果，根据结果和计划再启动下一个操作，例如`ReactJsonAgent`。

## Agent 的类型

### CodeAgent

该 Agent 有一个确定的规划和执行步骤，会生成 Python 代码来一次性执行所有的操作。它能够为其工具处理不同的输入和输出类型，因此它是多模态任务的推荐选择。

### ReactJsonAgent

这是解决推理型任务的首选代理，因为[`ReAct框架`（Yao等人，2022年）](https://huggingface.co/papers/2210.03629)使 Agent 在执行后会观察之前的结果再决定下一步，使得每一步都建立在有效思考的基础上。

我们实现了两个版本的`ReactJsonAgent`：
- `ReactJsonAgent` 在其输出中将工具调用生成为 JSON。
- `ReactCodeAgent` 是一种新型的 ReactJsonAgent，它能够通过工具调用生成代码的二进制表示形式(Blob)，这对于具有强大编码能力的 LLMs 非常有效。

阅读[将开源 LLMs 作为 LangChain Agents ](https://huggingface.co/blog/open-source-llms-as-agents)博客文章，了解更多关于 ReAct Agent 的信息。

![Agent_ManimCE](../../resources/show/gif/Agent_ManimCE.gif)

![ReAct](../../resources/show/ReAct.png)

下面展示的是 ReactCodeAgent 如何解决问题的过程和输出内容：

In [ ]:
agent.run(
    "How many more blocks (also denoted as layers) in BERT base encoder than the encoder from the architecture proposed in Attention is All You Need?",
)

## 如何构建 Agent？
